# 動画をテキスト化するツールです。
mp3をテキスト化する場合→①②を実行
<br>
youtubeの動画をテキスト化する場合①③を実行
<br>
mp4をアップロードボタンに入れた場合でも変換はできますが、
<br>
google colabの仕様により、アップロードに非常に時間がかかります。
<br>2023/11/08 whisper v3に対応しました。

<h2>
①google driveの読み込み
</h2>
<h3>
google driveに文字起こししたテキストを保存するのに使います。
<br>
ポップアップが出たら、自身のgoogleアカウントでログインし、許可してください。
<br>
また、実行すると音声AIのwhisper v3が自動的にインストールされます。


In [ ]:
#ドライブへ保存するためのコードです。はじめに使用して下さい。
from google.colab import drive
drive.mount('/content/drive')
! pip install -U openai-whisper
import whisper
model = whisper.load_model("large")

#②mp3をテキスト化
<h3>
再生ボタンを押すと表示される下の緑色の<br>
「ファイルを選択」ボタンからmp3ファイルをアップロードしてください。
<br>
最終的にgoogle driveにテキストファイルが保存されます。

<br>
この処理は時間がかかる場合があります。特に長時間の動画は相当の時間がかかります。


In [ ]:
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
import os
import time
import whisper

# ffmpegのインストール
get_ipython().system('pip install ffmpeg')
import ffmpeg
def convert_mp4_to_mp3(input_path, output_path):
    (
        ffmpeg
        .input(input_path)
        .output(output_path)
        .run()
    )

def upload_and_convert(change):
    uploaded = files.upload()
    for fn in uploaded.keys():
        file_path = '/content/' + fn
        print('アップロードされたファイル:', fn)

        # mp4かmp3か判別
        if fn.endswith('.mp4'):
            mp3_path = file_path.replace('.mp4', '.mp3')
            convert_mp4_to_mp3(file_path, mp3_path)  # ここで変換関数を呼び出す
            file_path = mp3_path  # 変換後のパスを更新
            print("mp3への変換が完了しました。")
        # Display the file path of the mp3
        print('File path:', file_path)

        #flush
        time.sleep(1)
        print("文字起こしを開始します。この動作にはしばらく時間がかかる場合があります。画面を閉じずにお待ちください", flush=True)

        # Convert audio to text if mp3 exists
        if os.path.exists(file_path):
            # Assuming model is loaded elsewhere
            result = model.transcribe(file_path)

            # Display the recognized text
            print(result["text"])

            # Extract the title from the filename
            title = os.path.splitext(fn)[0]

            # Save the recognized text to a file in Google Drive
            output_path = f'/content/drive/MyDrive/{title}.txt'
            with open(output_path, 'w') as file:
                file.write(result["text"])

            # テキストファイルの内容を修正するロジック
            if os.path.exists(output_path):
                with open(output_path, "r", encoding="utf-8") as file:
                    content = file.read()

                # ピリオドの後に改行を追加
                updated_content = content.replace('。', '。\n')

                # 変更した内容を同じファイルに書き込む
                with open(output_path, "w", encoding="utf-8") as file:
                    file.write(updated_content)

            print(f"テキストファイルは以下の場所に保存されました: {output_path}")
        else:
            print(f"指定されたファイルが存在しません: {file_path}")

# アップロードボタンのスタイル設定

button_style = widgets.ButtonStyle(
    button_color='green',  # Background color
    font_weight='bold',        # Font weight
    width='400px',             # Button width
    height='150px'              # Button height
)

# mp4 mp3アップロードボタン
upload_button = widgets.Button(description="mp3アップロードボタン", style=button_style)
upload_button.on_click(upload_and_convert)
display(upload_button)

# ③Youtubeの動画をテキスト化する
<h3>右下の欄にyoutubeのURLを入力して再生ボタン▶️を押せばテキストファイルがgoogle driveに保存されます。<br>
必ず①を先に実行してから押して下さい。

In [ ]:
get_ipython().system('pip install ffmpeg')
import whisper
#youtube動画をテキスト化
youtube_url = "https://www.youtube.com/watch?v=hoge" #@param {type:"string"} { display-mode: "form" }
!pip install pytube
from pytube import YouTube

#@title TITLE

try:
   video = YouTube(youtube_url)
   stream = video.streams.filter(only_audio=True).first()
   stream.download(filename=f"{video.title}.mp3")
   print("mp3のダウンロードが完了しました。")
except KeyError:
   print("エラーが発生しました。URLが間違っている可能性があります。")

#ここからaudio to text
import os

# Assuming model is already loaded as shown in your previous code
mp3_path = f"/content/{video.title}.mp3"

# ファイルが存在するか確認
if os.path.exists(mp3_path):
    # 音声をテキストに変換
    result = model.transcribe(mp3_path)

    # 結果を表示
    print(result["text"])

    # Save the recognized text to a file in Google Drive
    output_path = f'/content/drive/MyDrive/{video.title}.txt'
    with open(output_path, 'w') as file:
        file.write(result["text"])

    print(f"テキストファイルは以下の場所にほぞんされました: {output_path}")
else:
    print(f"指定されたファイルが存在しません: {mp3_path}")

#txtに改行を加える
if os.path.exists(output_path):
    with open(output_path, "r", encoding="utf-8") as file:
        content = file.read()

    # ピリオドの後に改行を追加
    updated_content = content.replace('。', '。\n')

    # 変更した内容を同じファイルに書き込む
    with open(output_path, "w", encoding="utf-8") as file:
        file.write(updated_content)

